In [1]:
import urbans

In [ ]:
# TODO: Chain rules to add for VMod and VPrepOrgMod
src_grammar = """S -> NPQ VP Qmark
S -> NPQ was Nominal Qmark
S -> NPQ did NPVmod Qmark
S -> was Nominal Vobl Qmark
S -> NPQ Vobl Qmark
S -> was Nominal Adj Qmark
S -> was NominalA NominalB Qmark
S -> did Nominal VP Qmark

NominalA -> Nominal
NominalB -> Nominal

NPVmod -> Nominal VMod
NPVmod -> Nominal VPrepOrgMod

VP -> V Nominal
VP -> was Vobl
VPT -> VT Nominal
VPT -> was VoblT 
VPMod -> VMod Nominal
VPMod -> was VoblMod

VPrep -> was VPrep
VPrep -> V by
VPrepT -> was VPrepT
VPrepT -> VT by
VPrepMod -> was VPrepMod
VPrepMod -> VMod by

VPrepOrgMod -> was VPrepOrgMod
VPrepOrgMod -> VPassOrgMod by

Vobl -> VPrep Nominal
VoblT -> VPrepT Nominal 
VoblMod -> VPrepMod Nominal

NPQ -> WhW Nominal 
NPQ -> WhW role caseO

commonNoun -> commonNoun RC

RC -> VoblMod
RC -> R VPMod
RC -> R NPVmod
RC -> whose role VPMod

VP -> VPT andVP
VP -> VPx andVP
VPx -> VPT punctVPT 
VPx -> VPx punctVPT 
andVP -> conj VP 
andVP -> punct conj VP
punctVPT -> punct VPT

VPMod -> VPT andVPMod
VPMod -> VPx andVPMod
VPx -> VPT punctVPT 
VPx -> VPx punctVPT 
andVPMod -> conj VPMod
andVPMod -> punct conj VPMod
punctVPT -> punct VPT

Vobl -> VoblT andVobl
Vobl -> Voblx andVobl 
Voblx -> VoblT punctVoblT 
Voblx -> Voblx punctVoblT 
andVobl -> conj Vobl 
andVobl -> punct conj Vobl
punctVoblT -> punct VoblT

VoblMod -> VoblT andVoblMod
VoblMod -> Voblx andVoblMod 
Voblx -> VoblT punctVoblT 
Voblx -> Voblx punctVoblT 
andVoblMod -> conj VoblMod 
andVoblMod -> punct conj VoblMod
punctVoblT -> punct VoblT


"""

In [2]:
src_to_target_grammar = {
    "S -> NPQ VP Qmark":"S -> NPQ が VP か Qmark", 
    "S -> NPQ was Nominal Qmark":"S -> Nominal は NPQ でしたか Qmark", # Q
    "S -> NPQ did NPVmod Qmark":"S -> NPVmod のは NPQ でしたか Qmark", # what did A write and B edit? -> A ga kaite, B ga hennshushita no ha nann deshitaka
    "S -> was Nominal Vobl Qmark":"S -> Nominal は Vobl か Qmark",
    "S -> NPQ Vobl Qmark":"NPQ が Vobl か Qmark", # same Q
    "S -> was Nominal Adj Qmark":"S -> Nominal は Adj でしたか Qmark",
    "S -> was NominalA NominalB Qmark":"S -> NominalA は NominalB でしたか Qmark",
    "S -> did Nominal VP Qmark":"S -> Nominal は VP か Qmark",

    # TODO: Chain rules to add for VMod and VPrepOrgMod
    "NPVmod -> Nominal VMod":"NPVmod -> Nominal が VMod",
    # todo: this rule basically follows VP, VPrepOrg finally derives as V
    "NPVmod -> Nominal VPrepOrgMod":"NPVmod -> Nominal を VPrepOrgMod", # VPrepOrg: passive in EN, active in JP 


    "VP -> V Nominal":"VP -> Nominal を V", # have lunch -> hirugohan wo taberu
    "VP -> was Vobl":"VP -> Vobl", # was done by him -> kare ni sareta
    "VPT -> VT Nominal":"VPT -> Nominal を VT",
    "VPT -> was VoblT":"VPT -> VoblT",
    "VPMod -> VMod Nominal":"VPMod -> Nominal を VMod",
    "VPMod -> was VoblMod":"VPMod -> VoblMod",

    "VPrep -> was VPrep":"VPrep -> VPrep",
    "VPrep -> V by":"VPrep -> V",
    "VPrepT -> was VPrepT":"VPrepT -> VPrepT",
    "VPrepT -> VT by":"VPrepT -> VT",
    "VPrepMod -> was VPrepMod":"VPrepMod -> VPrepMod",
    "VPrepMod -> VMod by":"VPrepMod -> VMod",

    "VPrepOrgMod -> was VPrepOrgMod":"VPrepOrgMod -> VPrepOrgMod",
    "VPrepOrgMod -> VPassOrgMod by":"VPrepOrgMod -> VPassOrgMod",

    "Vobl -> VPrep Nominal":"Vobl -> Nominal に VPrep", # done by nominal -> nominal ni sareta
    "VoblT -> VPrepT Nominal":"VoblT -> Nominal に VPrepT",
    "VoblMod -> VPrepMod Nominal":"VoblMod -> Nominal に VPrepMod",
    
    "NPQ -> WhW Nominal":"NPQ -> WhW Nominal", # which city -> dono machi
    # TODO: caseO -> of ... : caseO ... no
    "NPQ -> WhW role caseO":"NPQ -> caseO WhW role", # which character of Alice -> alisu no dono kyarakuta

    "commonNoun -> commonNoun RC":"commonNoun -> RC commonNoun", # people whose role was... -> yaku ga... hito

    "RC -> VoblMod":"RC -> VoblMod", # people attacked by... -> ...ni kougekisareta hito 
    "RC -> R VPMod":"RC -> VPMod", # people that eat... -> ...wo taberu hito
    "RC -> R NPVmod":"RC -> NPVmod", # people that you like -> anata ga sukina hito
    "RC -> whose role VPMod":"RC -> role が VPMod", # todo: role GA -> ...

    "VP -> VPT andVP":"VP -> VPT andVP",
    "VP -> VPx andVP":"VP -> VPx andVP",
    "VPx -> VPT punctVPT":"VPx -> VPT punctVPT",
    "VPx -> VPx punctVPT":"VPx -> VPx punctVPT",
    "andVP -> conj VP":"andVP -> conj VP",
    "andVP -> punct conj VP":"andVP -> punct conj VP",
    "punctVPT -> punct VPT":"punctVPT -> punct VPT",

    # VPMod -> VPT andVPMod
    # VPMod -> VPx andVPMod
    # VPx -> VPT punctVPT 
    # VPx -> VPx punctVPT 
    # andVPMod -> conj VPMod
    # andVPMod -> punct conj VPMod
    # punctVPT -> punct VPT

    # Vobl -> VoblT andVobl
    # Vobl -> Voblx andVobl 
    # Voblx -> VoblT punctVoblT 
    # Voblx -> Voblx punctVoblT 
    # andVobl -> conj Vobl 
    # andVobl -> punct conj Vobl
    # punctVoblT -> punct VoblT

    # VoblMod -> VoblT andVoblMod
    # VoblMod -> Voblx andVoblMod 
    # Voblx -> VoblT punctVoblT 
    # Voblx -> Voblx punctVoblT 
    # andVoblMod -> conj VoblMod 
    # andVoblMod -> punct conj VoblMod
    # punctVoblT -> punct VoblT



}

special cases: like -> ga suki

In [ ]:
VPMod -> VPT andVPMod
VPMod -> VPx andVPMod
VPx -> VPT punctVPT 
VPx -> VPx punctVPT 
andVPMod -> conj VPMod
andVPMod -> punct conj VPMod
punctVPT -> punct VPT